# Защита персональных данных клиентов

Для страховой компании «Хоть потоп» требуется методом преобразования данных защитить личную информацию клиентов, чтобы по ним было сложно восстановить персональную информацию.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

Подключим необходимые библиотеки для проекта

In [1]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

Загрузим файл с данными и выведем на экран первые 5 строк для ознакомления

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Есть количество членов семьи которые начинаются с нуля, предположительно это означает следующее:  
- `0` Не замужем/не женат
- `1` Клиент состоит в браке
- а `2+` это дети

Изучим общую информацию о датафрейме

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Датафрейм с данными включает в себя 5 количественных признаков: 4 - описательных признака (`Пол`, `Возраст`, `Зарплата`, `Члены семьи`), 1 - целевой признак (`Страховые выплаты`).  
Количество записей 5000, пропуски отсутствуют. Проверим на дубликаты на следующих шагах.

А пока для удобства работы с данными, переименуем и изменим названия колонок, удалим пропуски в названиях

In [4]:
data.rename(columns = {'Пол' : 'sex',
                       'Возраст' : 'age',
                       'Зарплата' : 'salary',
                       'Члены семьи' : 'family_members',
                       'Страховые выплаты' : 'insurance_benefits'}, inplace = True)

Заменим тип данных для признаков `возраст` и `заработная плата`.

In [5]:
features = ['age', 'salary']
data[features] = data[features].astype('int')

Сдери данных могут быть дубликаты,проверим датафрейм на их наличие.

In [6]:
data.duplicated().sum()

153

Обнаружили `153` дубликата, удалим и применим изменения

In [7]:
data.drop_duplicates(inplace = True)

При помощи метода `.describe()` посмотрим на статистическую информацию

In [8]:
data.describe()

,sex,age,salary,family_members,insurance_benefits
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811223,1.203425,0.152259
std,0.500049,8.487995,9972.952441,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Еще проверим есть ли корреляция признаков в данных.

In [9]:
data.corr()

,sex,age,salary,family_members,insurance_benefits
sex,1.000000,0.001953,0.015456,-0.007315,0.011565
age,0.001953,1.000000,-0.017386,-0.009064,0.654964
salary,0.015456,-0.017386,1.000000,-0.031687,-0.013123
family_members,-0.007315,-0.009064,-0.031687,1.000000,-0.039303
insurance_benefits,0.011565,0.654964,-0.013123,-0.039303,1.000000


Между возрастом клиентов и величиной страховых выплат наблюдается положительная корреляция.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии **не** изменится.

**Обоснование:** <br>
Обозначим новую матрицу признаков как $X'$. При умножении на произвольную обратимую матрицу она принимает следующий вид:

$$X' = XP$$

Тогда новый вектор весов можно выразить как:

$$w' = ((XP)^T(XP))^{-1}(XP)^Ty = (P^T X^T X P)^{-1} P^T X^T y = P^{-1} (X^T X) ^ {-1} (P^T)^{-1} P^T X^T y = P^{-1}w$$

Новые предсказания принимают следующий вид:

$$a' = X'w' = X P P^{-1} w = Xw = a$$

Поскольку в выражении для новых предсказаний произвольная матрица $P$ умножается на обратную ей матрицу (что дает в результате единичную матрицу), конечные предсказания и, как следствие, качество линейной регрессии не изменится.

## Алгоритм преобразования

**Алгоритм**

В целях защиты данных и сохранения качества линейной регрессии умножим признаки в датасете на обратимую матрицу $P$. 
- Сначала сгенерируем случайную матрицу <br>
- А после проверим сгенерированную матрицу на обратимость 

**Обоснование**

После теоретического доказательства мы убедились, что шифрование признаков с помощью умножения на обратную матрицу не влияет на качество линейной регрессии. Поэтому воспользуемся этим методов для шифрования признаков.

---

С использованием `np.random.randint` сгенерируем квадратную матрицу 4*4, поскольку в датасете у нас 4 объясняющих признака.

In [10]:
random_P = np.random.randint(100, size = (4,4))
random_P

array([[40, 33, 38, 31],
       [46, 29, 35,  4],
       [50, 82, 21, 72],
       [60, 81, 58, 46]])

Создадим обратную матрицу для проверки обратимости.

In [11]:
np.linalg.inv(random_P)

array([[ 0.00595464,  0.04552892,  0.01945206, -0.03841866],
       [-0.04780011,  0.00054318,  0.00097773,  0.03063552],
       [ 0.0269283 , -0.02899919, -0.02719853,  0.02694595],
       [ 0.04244976, -0.02377784,  0.00719989, -0.01607007]])

Разделим датасет на `features` и `target`

In [12]:
features = data.drop('insurance_benefits', axis = 1)
target = data['insurance_benefits']

Проверим размеры

In [13]:
print(features.shape)
print(target.shape)

(4847, 4)
(4847,)


Рассчитаем веса для признаков на исходных данных

In [14]:
w = np.linalg.inv(features.T.dot(features)).dot(features.T).dot(target)

Рассчитаем предсказания по признакам

In [15]:
a = features @ w

Произведем преобразование признаков

In [16]:
transformed_features = features @ random_P

Рассчитаем веса для преобразованных признаков, умноженных на сгенерированную матрицу

In [17]:
new_w = np.linalg.inv(transformed_features.T.dot(transformed_features)).dot(transformed_features.T).dot(target)

Рассчитаем предсказания для преобразованных данных

In [18]:
new_a = transformed_features @ new_w

Посмотрим на расхождения предсказаний

In [19]:
difference_sum = (a - new_a).sum()
print(f"{difference_sum:.17f}")

0.00292600018150280


Сумма расхождений незначительная - следовательно, качество линейной регрессии не изменится.

## Проверка алгоритма

Обучим модель `LinearRegression()` и сравним значение `MSE` и `R2` до и после преобразования признаков.
Обучим модель линейной регрессии на исходных данных.

In [20]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)

In [21]:
mse = mean_squared_error(target, predictions)
r2 = r2_score(target, predictions)

In [22]:
print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 0.1252726382276536
R2: 0.4302010046633359


Обучим модель линейной регрессии на новых данных

In [23]:
model = LinearRegression()
model.fit(transformed_features, target)
predictions = model.predict(transformed_features)

In [24]:
mse = mean_squared_error(target, predictions)
r2 = r2_score(target, predictions)

In [25]:
print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 0.12527263822765183
R2: 0.4302010046633439


Метрики `MSE` и `R2` после преобразования признаков не изменились.

## Вывод

Во время выполнения проекта были выполнены сделаны такие шаги: 

- Данные были загружены и изучены 
- Теоретически доказано, что при умножении признаков на обратимую матрицу качество регрессии не меняется 
- Выполнена трансформация данных о клиентах компании «Хоть потоп», в результате обучения модели линейной регрессии было продемонстрировано, что ее качество не меняется после умножения признаков на обратимую матрицу.